In [3]:
import duckdb as db

In [4]:
def sql(query, database = "../data/database.duckdb"):
    con = db.connect(database)
    res = con.execute(query).df()
    con.close()
    return res

In [9]:
urls = {
    'name_basics' : 'https://datasets.imdbws.com/name.basics.tsv.gz',
    'title_basics' : 'https://datasets.imdbws.com/title.basics.tsv.gz',
    'title_crew' : 'https://datasets.imdbws.com/title.crew.tsv.gz',
    'title_ratings' : 'https://datasets.imdbws.com/title.ratings.tsv.gz',
    'title_akas' : 'https://datasets.imdbws.com/title.akas.tsv.gz',
    'tmdb' : '../data/tmdb_full.csv'
}

for name, url in urls.items():
    if name == 'title_basics':
        sql(
            f"""
                create or replace table {name} as
                select * from read_csv_auto('{url}')
                where titletype = 'movie'
            """
        )
    else:
        sql(
            f"""
                create or replace table {name} as
                select * from read_csv_auto('{url}')
            """
        )


In [11]:
sql(
            f"""
                create or replace table title_principals as
                select * from read_csv_auto('https://datasets.imdbws.com/title.principals.tsv.gz')
            """
        )

,Count
0,92549544


In [12]:
sql(
    """
    show tables

"""
)

,name
0,name_basics
1,title_akas
2,title_basics
3,title_crew
4,title_principals
5,title_ratings
6,tmdb
